In [1]:
import tensorflow as tf
from config import Config

config = Config()

def parse(record):
    feature_set = { 'theta':tf.io.VarLenFeature(dtype=tf.float32),
               'image':tf.io.FixedLenFeature([], tf.string,default_value='')
           }

    #tf.io.FixedLenFeature([10], tf.float32, default_value=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    features = tf.io.parse_single_example(record, features=feature_set)
    image_data = features['image']
    image = decodeImg(image_data)
    theta = features['theta']
    return image,theta

def decodeImg(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    # convert to [0, 1].
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    #resize dell'immagine
    encoder_img_size = config.ENCODER_INPUT_SHAPE[:2]
    image_resize = tf.image.resize(image, encoder_img_size, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    # Normalize image to [-1, 1]
    image_final = tf.subtract(image_resize, 0.5)
    image = tf.multiply(image_final, 2.0)
    return image
    
class ExceptionHandlingIterator:
    """This class was introduced to avoid tensorflow.python.framework.errors_impl.InvalidArgumentError
        thrown while iterating over the zipped datasets.

        One assumption is that the tf records contain one wrongly generated set due to following error message:
            Expected begin[1] in [0, 462], but got -11 [[{{node Slice}}]] [Op:IteratorGetNextSync]
    """

    def __init__(self, iterable):
        self._iter = iter(iterable)

    def __iter__(self):
        return self

    def __next__(self):
        try:
            return self._iter.__next__()
        except StopIteration as e:
            raise e
        except Exception as e:
            print(e)
            return self.__next__()

    

In [2]:
#records = tf.data.TFRecordDataset("Run2.tfrecords")
import tensorflow as tf
import tqdm
tf_records = "Run2.tfrecords"
dataset = tf.data.TFRecordDataset(tf_records)
samples = 120577
total = int(samples/ 64)
print(total)
dataset = dataset.map(parse)
dataset = dataset.shuffle(10000, seed=config.SEED, reshuffle_each_iteration=True)
dataset = dataset.batch(batch_size=64, drop_remainder=True)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
print(dataset)


#for data in tqdm(dataset, total=total, position=0, desc='training'):
    #print(data[0])


1884
<PrefetchDataset shapes: ((64, 224, 224, 3), (64, None)), types: (tf.float32, tf.float32)>
